In [1]:
# load library
library(tidyverse)
library(lubridate)

# set working directory
setwd("~/Desktop")
options(digits = 10)

# load dataset
dt <- read_csv("NYPD_Motor_Vehicle_Collisions.csv")

# explore dataset
head(dt)
colnames(dt)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Parsed with column specification:
cols(
  .default = col_character(),
  TIME = col_time(format = ""),
  `ZIP CODE` = col_integer(),
  LATITUDE = col_double(),
  LONGITUDE = col_double(),
  `NUMBER OF PERSONS INJURED` = col_integer(),
  `NUMBER OF PERSONS KILLED` = col_integer(),
  `NUMBER OF PEDESTRIANS INJURED` = col_integer(),
  `NUMBER OF PEDESTRIANS KILLED` = col_integer(),
  `NUMBER OF CYCLIST INJURED` = col_integer(),
  `NUMBER OF CYCLIST KILLED` = col_integer(),
  `NUMBER OF MOTORIST INJURED` = col_integer(),

DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,⋯,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
04/30/2019,00:00:00,BROOKLYN,11222,40.727184,-73.950700,"(40.727184, -73.9507)",NA,NA,236 ECKFORD STREET,⋯,Unspecified,NA,NA,NA,4123208,Sedan,Station Wagon/Sport Utility Vehicle,NA,NA,NA
04/30/2019,00:00:00,MANHATTAN,10075,40.776318,-73.962135,"(40.776318, -73.962135)",EAST 79 STREET,MADISON AVENUE,NA,⋯,Unspecified,NA,NA,NA,4123128,Station Wagon/Sport Utility Vehicle,Ambulance,NA,NA,NA
04/30/2019,00:00:00,QUEENS,11354,40.763283,-73.830710,"(40.763283, -73.83071)",NA,NA,137-17 NORTHERN BOULEVARD,⋯,Unspecified,NA,NA,NA,4124069,Sedan,NA,NA,NA,NA
04/30/2019,00:00:00,QUEENS,11412,40.705240,-73.775080,"(40.70524, -73.77508)",LIBERTY AVENUE,DUNKIRK STREET,NA,⋯,Unspecified,NA,NA,NA,4123202,COMMU,Sedan,NA,NA,NA
04/30/2019,00:00:00,NA,NA,40.593525,-73.996280,"(40.593525, -73.99628)",BELT PARKWAY,NA,NA,⋯,Unspecified,NA,NA,NA,4123344,Station Wagon/Sport Utility Vehicle,Sedan,NA,NA,NA
04/30/2019,00:00:00,NA,NA,40.744240,-73.847170,"(40.74424, -73.84717)",GRAND CENTRAL PKWY,NA,NA,⋯,Unspecified,NA,NA,NA,4123160,Sedan,Station Wagon/Sport Utility Vehicle,NA,NA,NA


[1] "DATE"                          "TIME"                         
 [3] "BOROUGH"                       "ZIP CODE"                     
 [5] "LATITUDE"                      "LONGITUDE"                    
 [7] "LOCATION"                      "ON STREET NAME"               
 [9] "CROSS STREET NAME"             "OFF STREET NAME"              
[11] "NUMBER OF PERSONS INJURED"     "NUMBER OF PERSONS KILLED"     
[13] "NUMBER OF PEDESTRIANS INJURED" "NUMBER OF PEDESTRIANS KILLED" 
[15] "NUMBER OF CYCLIST INJURED"     "NUMBER OF CYCLIST KILLED"     
[17] "NUMBER OF MOTORIST INJURED"    "NUMBER OF MOTORIST KILLED"    
[19] "CONTRIBUTING FACTOR VEHICLE 1" "CONTRIBUTING FACTOR VEHICLE 2"
[21] "CONTRIBUTING FACTOR VEHICLE 3" "CONTRIBUTING FACTOR VEHICLE 4"
[23] "CONTRIBUTING FACTOR VEHICLE 5" "UNIQUE KEY"                   
[25] "VEHICLE TYPE CODE 1"           "VEHICLE TYPE CODE 2"          
[27] "VEHICLE TYPE CODE 3"           "VEHICLE TYPE CODE 4"          
[29] "VEHICLE TYPE CODE 5"

# What is the total number of persons injured in the dataset (up to December 31, 2018?)

In [2]:
dt <- dt %>% mutate(DATE = mdy(DATE)) # re-format date
dt %>%
  filter(DATE < "2019-01-01") %>%
  summarise(sum(`NUMBER OF CYCLIST INJURED`))

sum(`NUMBER OF CYCLIST INJURED`)
29182


# What proportion of collisions in 2016 resulted in injury or death of a cyclist?

In [3]:
dt2016 <- dt %>%
  filter(DATE < "2017-01-01" & DATE > "2015-12-31") %>%
  mutate(victim = `NUMBER OF CYCLIST KILLED` + `NUMBER OF CYCLIST INJURED`, 
            victim = ifelse(victim == 0, "no", 'yes'))
mean(dt2016$victim == "yes")

[1] 0.02165483687

# Obtain the number of vehicles involved in each collision in 2016. Group the collisions by zip code and compute the sum of all vehicles involved in collisions in each zip code, then report the maximum of these values.

In [4]:
table(dt$`CONTRIBUTING FACTOR VEHICLE 5`, useNA = "always")
# Most collisions don't involve fifth car.
dt2016 <- dt2016 %>%
  mutate(`CONTRIBUTING FACTOR VEHICLE 1` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 1`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 2` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 2`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 3` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 3`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 4` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 4`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 5` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 5`), 0, 1),
         cars = `CONTRIBUTING FACTOR VEHICLE 1` + `CONTRIBUTING FACTOR VEHICLE 2` + 
           `CONTRIBUTING FACTOR VEHICLE 3` + `CONTRIBUTING FACTOR VEHICLE 4` +
           `CONTRIBUTING FACTOR VEHICLE 5`)
# Here I assume if "CONTRIBUTING FACTOR VEHICLE" 1-5 is coded as NA, then there is no car involved; 
# otherwise a car is involved.
dt2016 %>%
  group_by(`ZIP CODE`) %>%
  summarise(n = sum(cars)) %>%
  arrange(desc(n))


               Aggressive Driving/Road Rage 
                                          1 
                        Alcohol Involvement 
                                          8 
                           Backing Unsafely 
                                          1 
                           Brakes Defective 
                                          1 
             Driver Inattention/Distraction 
                                         37 
                        Driver Inexperience 
                                          8 
                            Drugs (illegal) 
                                          2 
                      Failure to Keep Right 
                                          2 
              Failure to Yield Right-of-Way 
                                          4 
                            Fatigued/Drowsy 
                                         41 
                                Fell Asleep 
                                          3 
         

ZIP CODE,n
NA,151758
11207,5243
11101,4082
11234,3839
10019,3737
11201,3703
11203,3679
11236,3628
10016,3627
10022,3594


# Question 4: Do winter driving conditions lead to more multi-car collisions? Compute the rate of multi car collisions as the proportion of the number of collisions involving 3 or more cars to the total number of collisions for each month of 2017. Calculate the chi-square test statistic for testing whether a collision is more likely to involve 3 or more cars in January than in May.

In [5]:
dt2017 <- dt %>%
  filter(DATE < "2018-01-01" & DATE > "2016-12-31") %>%
  mutate(date = str_extract(DATE, "\\d+\\-\\d+")) %>%
  mutate(`CONTRIBUTING FACTOR VEHICLE 1` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 1`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 2` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 2`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 3` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 3`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 4` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 4`), 0, 1),
         `CONTRIBUTING FACTOR VEHICLE 5` = ifelse(is.na(`CONTRIBUTING FACTOR VEHICLE 5`), 0, 1),
         cars_ttl = `CONTRIBUTING FACTOR VEHICLE 1` + `CONTRIBUTING FACTOR VEHICLE 2` + 
           `CONTRIBUTING FACTOR VEHICLE 3` + `CONTRIBUTING FACTOR VEHICLE 4` +
           `CONTRIBUTING FACTOR VEHICLE 5`,
         cars_multi = `CONTRIBUTING FACTOR VEHICLE 3` + `CONTRIBUTING FACTOR VEHICLE 4` +
           `CONTRIBUTING FACTOR VEHICLE 5`) %>%
  group_by(date) %>%
  summarise(cars_ttl = sum(cars_ttl), cars_multi = sum(cars_multi), ratio = cars_multi / cars_ttl)
jan_may <- rbind(dt2017[1, 2:3], dt2017[5, 2:3])
chisq.test(jan_may)


	Pearson's Chi-squared test with Yates' continuity correction

data:  jan_may
X-squared = 3.5038841, df = 1, p-value = 0.06122508


# What proportion of all collisions in 2016 occured in Brooklyn? Only consider entries with a non-null value for BOROUGH.

In [6]:
data <- dt2016 %>%
  filter(!is.na(BOROUGH)) %>%
  group_by(BOROUGH) %>%
  summarise(n = n())
pct <- prop.table(data$n)
names(pct) <- data$BOROUGH
pct

BRONX      BROOKLYN     MANHATTAN        QUEENS STATEN ISLAND 
0.14130916967 0.30961980064 0.23533479463 0.26996894734 0.04376728772

# For each borough, compute the number of accidents per capita involving alcohol in 2017. Report the highest rate among the 5 boroughs. Use populations as given by https://en.wikipedia.org/wiki/Demographics_of_New_York_City.

In [7]:
alcohol <- dt %>%
  filter(DATE < "2018-01-01" & DATE > "2016-12-31" & !is.na(BOROUGH) & 
           (str_detect(`CONTRIBUTING FACTOR VEHICLE 1`, "Alcohol") |
              str_detect(`CONTRIBUTING FACTOR VEHICLE 2`, "Alcohol") |
              str_detect(`CONTRIBUTING FACTOR VEHICLE 3`, "Alcohol") |
              str_detect(`CONTRIBUTING FACTOR VEHICLE 4`, "Alcohol") |
              str_detect(`CONTRIBUTING FACTOR VEHICLE 5`, "Alcohol"))) %>%
  group_by(BOROUGH) %>%
  summarise(accident_num = n())
capita <- c(1471160, 2648771, 1664727, 2358582, 479458)
alcohol <- cbind(alcohol, capita)
alcohol %>%
  mutate(rate = accident_num / capita) %>%
  arrange(desc(rate))

BOROUGH,accident_num,capita,rate
BROOKLYN,602,2648771,0.0002272752156
QUEENS,512,2358582,0.0002170795843
STATEN ISLAND,100,479458,0.0002085688423
BRONX,274,1471160,0.0001862475869
MANHATTAN,259,1664727,0.0001555810652


# Consider the total number of collisions each year from 2013-2018. Is there an apparent trend? Fit a linear regression for the number of collisions per year and report its slope.

In [8]:
years <- dt %>%
  mutate(year = str_extract(DATE, "\\d+")) %>%
  filter(year %in% (2013:2018)) %>%
  group_by(year) %>%
  summarise(n = n())
model <- lm(year ~ n, data = years)
summary(model)


Call:
lm(formula = year ~ n, data = years)

Residuals:
          1           2           3           4           5           6 
-0.24969671  0.43109723 -0.18671287 -0.86459087 -0.03244803  0.90235125 

Coefficients:
                Estimate   Std. Error   t value   Pr(>|t|)    
(Intercept) 1.984987e+03 5.246165e+00 378.36921 2.9273e-10 ***
n           1.387249e-04 2.381824e-05   5.82431  0.0043281 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6793127 on 4 degrees of freedom
Multiple R-squared:  0.8945221,	Adjusted R-squared:  0.8681526 
F-statistic: 33.92264 on 1 and 4 DF,  p-value: 0.004328148


# We can use collision locations to estimate the areas of the zip code regions. Represent each as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude. For collisions in 2017, estimate the number of collisions per square kilometer of each zip code region. Considering zipcodes with at least 1000 collisions, report the greatest value for collisions per square kilometer. Note: Some entries may have invalid or incorrect (latitude, longitude) coordinates. Drop any values that are invalid or seem unreasonable for New York City.
*I don't know how to connect square kilometer to coordinates...*